# ROOT file format example

To run this example, you will need `uproot`, which is another SciKit-HEP library.

In [ ]:
import numpy as np
import boost_histogram as bh
import uproot
import os
import matplotlib.pyplot as plt

[ROOT](https://root.cern/) is a modular scientific software toolkit used in High Energy Physics. (HEP) The ROOT file format is used to store almost all HEP data. This notebook will illustrate one method for converting to/from the ROOT file format using uproot, a Python implementation of a ROOT file reader and writer.

For more complicated histograms, you may need Aghast and PyROOT, but that is a much heaver dependency, and is covered [in a separate tutorial](https://scikit-hep.org/scikit-hep-tutorials).

Start by making a 1D histogram:

In [ ]:
h = bh.Histogram(bh.axis.Regular(15, -3, 3))
h.fill(np.random.normal(size=1_000_000))

In [ ]:
root_file = uproot.recreate("demo_uproot_file.root")

# Uproot automatically converts NumPy style tuples
root_file["hist"] = h.to_numpy()

If you want to save and load the ROOT histogram, use [uproot](https://github.com/scikit-hep/uproot) to read and write:

In [ ]:
root_file2 = uproot.open("demo_uproot_file.root")
uproot_hist = root_file2['hist']
uproot_hist.show()

This uproot histogram can be converted to NumPy with `.numpy()`:

In [ ]:
h = bh.Histogram(bh.axis.Variable(uproot_hist.edges))
h[:] = uproot_hist.allvalues
plt.bar(h.axes[0].centers, h.view(), width=h.axes[0].widths);

We could use a `Weight()` storage and read both `allvalues` and `allvariances` in, as well, since ROOT histograms can sometimes have this enabled.

Finally, let's clean up after ourselves:

In [ ]:
if os.path.exists("demo_uproot_file.root"):
    os.remove("./demo_uproot_file.root")